파이토치에서 모델은 module 이라는 클래스를 기반으로 만들어짐

module을 subclassing해서 모델을 만드는방법이 많이 사용 됨


https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

기본 예제에 나오는 모델 구성 예시를 살펴보자

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

gpu 가능한지 확인, mps는 애플 gpu

In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


위의 if 문은 Conditional Expression이라고 하면 if else문을 간단히 아래같은 방식으로 작성한 것입니다
```
value_if_true if condition else value_if_false
```

nn.Module을 subclassing하여 __init__과 forward를 재작성합니다

__init__ 함수에는 레이어 구성 요소를 작성하고,
forward에는 연산을 작성합니다.

In [4]:


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()  # super()는 superclass(또는 부모 클래스)를 가리킵니다. 따라서 nn.Module의 __init__함수를 실행합니다.
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(   # Sequantial을 일부분으로 활용할 수 있습니다.
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
for m in model.children():
  print(m)

Flatten(start_dim=1, end_dim=-1)
Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=10, bias=True)
)


torchinfo를 통해서 모델의 좀더 많은 정보를 살펴봅시다

https://github.com/tyleryep/torchinfo

In [10]:
!pip install torchinfo  # !는 os 명령어를 사용할 수 있게해줍니다. Colab의 os는 ubuntu입니다.

In [11]:
from torchinfo import summary

In [12]:
summary(model, input_size=(16, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [16, 10]                  --
├─Flatten: 1-1                           [16, 784]                 --
├─Sequential: 1-2                        [16, 10]                  --
│    └─Linear: 2-1                       [16, 512]                 401,920
│    └─ReLU: 2-2                         [16, 512]                 --
│    └─Linear: 2-3                       [16, 512]                 262,656
│    └─ReLU: 2-4                         [16, 512]                 --
│    └─Linear: 2-5                       [16, 10]                  5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
Total mult-adds (M): 10.72
Input size (MB): 0.05
Forward/backward pass size (MB): 0.13
Params size (MB): 2.68
Estimated Total Size (MB): 2.86